In [109]:
import pandas as pd
from pathlib import Path
from os import listdir
from os.path import isfile, join
import numpy as np
from datetime import timedelta
"""
Prduces time-series from raw data, 
cleaning outliers and formatting data with bond information
such as coupon rates, maturity, face value, etc...
for the estimation of the yield curve
"""

def concat_brazil_bonds(datapath, saveto, filename):

    onlyfiles = [f for f in listdir(datapath) if '.xls' in f]

    files_info = {}
    for f in onlyfiles:
        xls = pd.ExcelFile(datapath / Path(f))
        files_info[f] = xls.sheet_names

    list_dfs = []

    for key, value in files_info.items():
        for sheet in value:
            df = pd.read_excel(datapath / Path(key), sheet_name = sheet)
            
            new_header = df.iloc[0] 
            df = df[1:] 
            df.columns = new_header 

            try:
                df['MidPrice'] = (df['PU Compra Manhã'] + df['PU Venda Manhã'])/2
            except:
                df['MidPrice'] = (df['PU Compra 9:00'] + df['PU Venda 9:00'])/2

            df = df[['Dia','MidPrice']]

            t = iter(sheet.replace('NTN-F ','').replace('LTN ','').replace('NTNF ',''))
            df['Maturity'] = '-'.join(a+b for a,b in zip(t, t))

            df = df.dropna()

            try:
                df['Dia'] = pd.to_datetime(df['Dia'], format='%d/%m/%Y')
            except:
                df['Dia'] = pd.to_datetime(df['Dia'], format='%d/%m/%yy')
            
            try:
                df['Maturity'] = pd.to_datetime(df['Maturity'], format='%d-%m-%y')
            except:
                df['Maturity'] = pd.to_datetime(df['Maturity'], format='%d-%m-%Y')

            df['BondName'] = sheet

            df['CouponRate'] = 0

            # fixed coupon bonds are always 10% coupon bonds
            if 'NTN-F' in sheet:
                df['CouponRate'] = 0.1
            if 'NTNF' in sheet:
                df['CouponRate'] = 0.1
                
            df['InstrumentPeriod'] = 2 # semi-annual coupon 

            list_dfs.append(df)


    hist = pd.concat(list_dfs)
    hist.columns = ['Date','MidPrice','Maturity', 'BondName','CouponRate','InstrumentPeriod']
    hist['MidPrice'] = pd.to_numeric(hist['MidPrice'])/10 # normalize to 100 face value 
    hist = hist[hist['MidPrice'] > 0]

    hist.to_csv(saveto / filename)

    return hist

def concat_italian_bonds(datapath, saveto, filename): 
    itbonds = pd.read_csv(datapath / 'IT_Bonds.csv')
    itbonds['Maturity'] = pd.to_datetime(itbonds['Maturity'], format = '%m/%d/%Y')

    historicalisins = pd.read_csv(datapath / "historical_isins_italy_with_couponrate.csv")

    historicalisins['CouponRate'] = historicalisins['CouponRate']/100
    historicalisins['CouponFreq'] = '6M'
    historicalisins['InstrumentPeriod'] = 2
    historicalisins ['BondName'] = historicalisins['Bond Name'].copy()
    del historicalisins['Bond Name']
    historicalisins['CouponType'] = 'Fixed rate'
    historicalisins['Maturity'] = pd.to_datetime(historicalisins['Maturity'], format='%d/%m/%Y')

    itbonds = itbonds[['ISIN', 'Maturity', 'CouponRate','CouponFreq', 'InstrumentPeriod', 'BondName','CouponType']]

    itbonds = pd.concat([itbonds, historicalisins])

    allisincheck = itbonds['ISIN'].unique().tolist() + historicalisins['ISIN'].unique().tolist() 
    print([x for x in allisincheck if x not in itbonds['ISIN'].unique().tolist()])


    itbondsdata = pd.read_csv(datapath / 'ITbondsdata.csv')
    historicalisinsdata = pd.read_csv(datapath / "Brunna-IT-bonds-matured.csv")

    itbondsdata = pd.concat([itbondsdata.set_index(['Timestamp']), historicalisinsdata.set_index(['Timestamp'])], axis = 1).sort_index().reset_index()
    itbondsdata = pd.DataFrame(itbondsdata.set_index(['Timestamp']).unstack()).reset_index()
    itbondsdata.columns = ['ISIN','Date','MidPrice']
    itbondsdata = itbondsdata.dropna().sort_values(by = ['Date'])
    itbondsdata = itbondsdata[itbondsdata['Date'] > '2002-01-01']

    allisincheckdata = itbondsdata['ISIN'].unique().tolist()
    missingbonds = [x for x in itbonds['ISIN'].unique().tolist() if x not in allisincheckdata]

    couponrate = itbonds.set_index(['ISIN']).to_dict()['CouponRate']
    couponfreq = itbonds.set_index(['ISIN']).to_dict()['InstrumentPeriod']
    mat = itbonds.set_index(['ISIN']).to_dict()['Maturity']
    BondName = itbonds.set_index(['ISIN']).to_dict()['BondName']
    CouponType = itbonds.set_index(['ISIN']).to_dict()['CouponType']

    itbondsdata['CouponRate'] = itbondsdata['ISIN'].map(couponrate)
    itbondsdata['InstrumentPeriod'] = itbondsdata['ISIN'].map(couponfreq)
    itbondsdata['Maturity'] = itbondsdata['ISIN'].map(mat)
    itbondsdata['BondName'] = itbondsdata['ISIN'].map(BondName)
    itbondsdata['CouponType'] = itbondsdata['ISIN'].map(CouponType)

    itbondsdata = itbondsdata.drop_duplicates(subset=['ISIN','Date']).dropna()

    itbondsdata.to_csv(saveto / filename)

    return itbondsdata

In [ ]:

local_drive = Path('C:/Users/torin/OneDrive')
file_path = Path('Desktop/Thesis/data/bonds/BR_raw')
datapath = (local_drive / file_path)
saveto = (local_drive / 'Desktop/Thesis/data/bonds/BR')

br_hist = concat_brazil_bonds(datapath, saveto, 'BR_FixedHistorical.csv')

In [110]:
local_drive = Path('C:/Users/torin/OneDrive')
file_path = Path('Desktop/Thesis/data/bonds/IT_raw')
datapath = (local_drive / file_path)
saveto = (local_drive / 'Desktop/Thesis/data/bonds/IT')

it_hist = concat_italian_bonds(datapath, saveto, 'IT_FixedHistorical.csv')

[]


In [ ]:
##################### SCRATCH #######################3

In [ ]:
it_hist['tenor'] = pd.to_datetime(it_hist['Maturity']) - pd.to_datetime(it_hist['Date'])
it_hist['tenor'] = pd.to_datetime(it_hist['Maturity']) - pd.to_datetime(it_hist['Date'])
it_hist['tenor'] = np.round(it_hist['tenor'] / timedelta(days=365), 0)
it_hist[it_hist.isnull().any(axis=1)]

In [ ]:
import plotly.express as px

# the Nov-2008 problem in Italy:
# it seems the only bonds available were 18y+ during that time 
# Svensson is probably not fitting a great curve there because of it 

df = it_hist[(it_hist.Date > '2008-10-01')&(it_hist.Date < '2008-12-31')].set_index(['Date'])

fig = px.line(df, x=df.index, y="MidPrice", title='Bonds', color='ISIN')

#print ( it_hist[(it_hist.Date > '2008-10-01')&(it_hist.Date < '2008-12-31')].Maturity.unique())

fig.show()